# Mutual information CNN - NIPS

- Author: Quentin Boyadjian, ETS
- Date: 15/02/20
- Input : DTD texture database
- Output : Mutual information through a trained model

In [ ]:
## Import libraries and dependencies
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pandas as pd # To create dataframes
import numpy as np # To manipulate arrays
from numpy import expand_dims

import os # To manipulate path
import cv2
from tqdm.notebook import tqdm # To produce progress bars
import pickle
import random
from random import sample 

from random import sample 
from pathlib import Path # To fix path compatibility issues Windows/linux

#a supprimer
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D
from matplotlib.ticker import FormatStrFormatter
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

import fnmatch
import math 

#from jupyter_client.manager import KernelManager

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
#from keras.callbacks import TensorBoard

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard

import random
from random import sample 
random.seed(1000)

from datetime import date # To include the date in the filenames
from datetime import datetime

from keras.models import Model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import csv
import ntpath

import seaborn as sns

from tqdm.notebook import tqdm # To produce progress bars adpted to Jupyter notebook

import keras
from keras.models import Sequential
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

import pickle

from pathlib import Path # To fix path compatibility issues Windows/linux


from sklearn.metrics.cluster import normalized_mutual_info_score

from warnings import simplefilter # import warnings filter
simplefilter(action='ignore', category=FutureWarning) # ignore all future warnings

# Define variables

projectDirectory=Path("C:/Users/AQ62270/201910_ccn/NIPS/") # Project path
inputDirectory=Path("0_data/dtd/images/") # Location of folders with pictures (in subfolders "nodefect" & "defect")
inputPath=os.path.join(projectDirectory,inputDirectory) # Create the full path to the input pictures

outputDirectory=Path("2_pipeline/store/") # Location of the serialized pictures
outputPath=os.path.join(projectDirectory,outputDirectory) # Create the full path to the output pictures
dtd=Path("C:/Users/AQ62270/201910_ccn/NIPS/0_data/dtd/images/")

IMG_SIZE = 227 # Set the value to resize the pictures, has to fit with the CNN input (227x227 is the default value)

DS = pd.DataFrame({"dataset" : []}) # Create a dataframe to run the script in batch mode
#DS = DS.append({"dataset" : "1"},ignore_index=True)
DS = DS.append({"dataset" : "2"},ignore_index=True)
#DS = DS.append({"dataset" : "3"},ignore_index=True)
#DS = DS.append({"dataset" : "4"},ignore_index=True)
#DS = DS.append({"dataset" : "5"},ignore_index=True)
#DS = DS.append({"dataset" : "6"},ignore_index=True)
#DS = DS.append({"dataset" : "7"},ignore_index=True)
#DS = DS.append({"dataset" : "8"},ignore_index=True)
#DS = DS.append({"dataset" : "9"},ignore_index=True)
#DS = DS.append({"dataset" : "10"},ignore_index=True)

## Dataset processing

- Input : unzipped DTD database available at https://www.robots.ox.ac.uk/~vgg/data/dtd/
- Output : Serialized dataset ready-to-learn (files = x3(train, validation, test) x2(image, label) x10(series))

In [ ]:
def get_size(start_path = '.'): # Check the folders size
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def create_training_data(): # Create the dataset for training (picture, label)
    for category in categories:  # For each folder, create a number for each category
        #path = os.path.join(DATA, category)  
        class_num = categories.index(category) 
        #for img in tqdm(os.path.join(category,os.path.split(lines)[1])):  # for each image in each folder, create array with label
        for line in lines:
            if category == os.path.split(line)[0]:
                try:
                    img_array = cv2.imread(os.path.join(Path(dtd,category,os.path.split(line)[1])))  # open the image in grayscale
                    #new_array = img_array[0:IMG_SIZE, 0:IMG_SIZE]
                    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize the image
                    new_array90 = np.rot90(new_array)
                    new_array180 = np.rot90(new_array90)
                    new_array270 = np.rot90(new_array180)
                    training_data.append([new_array, class_num])  # add it to the training dataset
                    training_data.append([new_array90, class_num])  # add it to the training dataset
                    training_data.append([new_array180, class_num])  # add it to the training dataset
                    training_data.append([new_array270, class_num])  # add it to the training dataset
                except Exception as e:  # To keep the output clean
                    pass
                            
i = 0
while i < DS.count()[0]: # For the number of entries in the DS dataframe:
    for serie in ["train", "test", "val"]:
        lines = []
        buffer=[]
        with open(os.path.join(Path('C:/Users/AQ62270/201910_ccn/NIPS/0_data/dtd/labels',serie+DS.iloc[0]["dataset"]+'.txt'))) as f:
            lines = [line.rstrip() for line in f]
            for image in lines:
                #print(os.path.join(Path(dtd,image)))
                #tmp = plt.imread(os.path.join(Path(dtd,image)))
                #plt.imshow(tmp)
                #plt.show()
                #print(os.path.split(image)[0])
                buffer.append(os.path.split(image)[0])
            categories = list( dict.fromkeys(buffer) )
            print("categories are :", categories)
            #DATA=os.path.join(inputPath, DS.iloc[i]["dataset"]) # Create a path to the cata
            
            training_data = [] # Create an empty matrix    
            create_training_data() # Create the dataset for training (picture, label)
            random.shuffle(training_data) # randomize the data
            datafeatures = [] # Create an empty vector for features training set
            trainDataLabel = [] # Create an empty vector for label training set
            for features,label in training_data:
                datafeatures.append(features)
                trainDataLabel.append(label)
            datafeatures = np.array(datafeatures).reshape(-1, IMG_SIZE, IMG_SIZE, 3) # reshape the data to fit with TensorFlow. (Grayscale=1, RGB=3)        
            
            datafeaturesName=os.path.join(DS.iloc[i]["dataset"][0] + "_"+serie+"datafeatures.pickle") # Generate a comprehensive name for serialized data
            datafeaturesPath=os.path.join(outputPath,datafeaturesName) # Create the full path to serialize the data
            pickle_out = open(datafeaturesPath,"wb") # Open path to serialize the data
            pickle.dump(datafeatures, pickle_out) # Serialize the data
            pickle_out.close()
            
            datalabelName=os.path.join(DS.iloc[i]["dataset"][0] + "_"+serie+"datalabel.pickle") # Generate a comprehensive name for serialized labels
            datalabelPath=os.path.join(outputPath,datalabelName)# Create the full path to serialize the labels
            pickle_out = open(datalabelPath,"wb") # Open path to serialize the labels
            pickle.dump(trainDataLabel, pickle_out) # Serialize the labels
            pickle_out.close()
    i += 1

## Train model

In [ ]:
## Parameters to specify
projectDirectory=Path("C:/Users/AQ62270/201910_ccn/NIPS/") # Project path
inputDirectory=Path("2_pipeline/store/") # Location of folders with pictures (in subfolders "nodefect" & "defect")
inputPath=Path(projectDirectory/inputDirectory) # Create the full path to the input pictures
#MODEL=Path(projectDirectory / "mod/") # Where to save the dataset
#LOG=Path(projectDirectory / "log/")
IMG_SIZE = 227 # has to fit with the value in "6_serialize_dataset"
outputDirectory=Path("2_pipeline/store/") # Location to save the models and logs
outputPath=Path(projectDirectory/outputDirectory) # Create the full path to the output pictures

# Define the design of experiments
DOE = pd.DataFrame({"dataset" : []}) # Create a dataframe to run the script in batch mode
#DOE = DOE.append({"dataset" : "1"},ignore_index=True)
DOE = DOE.append({"dataset" : "2"},ignore_index=True)
#DOE = DOE.append({"dataset" : "3"},ignore_index=True)
#DOE = DOE.append({"dataset" : "4"},ignore_index=True)
#DOE = DOE.append({"dataset" : "5"},ignore_index=True)
#DOE = DOE.append({"dataset" : "6"},ignore_index=True)
#DOE = DOE.append({"dataset" : "7"},ignore_index=True)
#DOE = DOE.append({"dataset" : "8"},ignore_index=True)
#DOE = DOE.append({"dataset" : "9"},ignore_index=True)
#DOE = DOE.append({"dataset" : "10"},ignore_index=True)

## Training the model
i = 0
while i < DOE.count()[0]:
    # File and folder locations
    trainDataLabel = pickle.load(open(Path(inputPath / os.path.join(DOE.iloc[i]["dataset"][0] + "_traindatalabel.pickle")),"rb"))
    valDataLabel = pickle.load(open(Path(inputPath / os.path.join(DOE.iloc[i]["dataset"][0] + "_valdatalabel.pickle")),"rb"))
    
    trainDataFeatures = pickle.load(open(Path(inputPath / os.path.join(DOE.iloc[i]["dataset"][0] + "_traindatafeatures.pickle")),"rb"))
    valDataFeatures = pickle.load(open(Path(inputPath / os.path.join(DOE.iloc[i]["dataset"][0] + "_valdatafeatures.pickle")),"rb"))

    ## Build the model: 
    model = Sequential()

    # 1st layer
    model.add(Conv2D(filters=96, 
                     input_shape=(IMG_SIZE,IMG_SIZE,3), 
                     kernel_size=(11,11), 
                     strides=(4,4), 
                     padding="valid", 
                     data_format="channels_last", 
                     name='conv1'
                    )
             )
    model.add(Activation('relu', name='activ1'))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding="valid", data_format="channels_last", name='pool1'))
    model.add(BatchNormalization(name='normal1'))

    # 2nd layer
    model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding="valid", data_format="channels_last", name='conv2'))
    model.add(Activation('relu', name='activ2'))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding="valid", data_format="channels_last", name='pool2'))
    model.add(BatchNormalization(name='normal2'))

    # 3rd layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="valid", data_format="channels_last", name='conv3'))
    model.add(Activation('relu', name='activ3'))
    model.add(BatchNormalization(name='normal3'))

    # 4th layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="valid", data_format="channels_last", name='conv4'))
    model.add(Activation('relu', name='activ4'))
    model.add(BatchNormalization(name='normal4'))

    # 5th layer
    model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding="valid", data_format="channels_last", name='conv5'))
    model.add(Activation('relu', name='activ5'))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding="valid", data_format="channels_last", name='pool3'))
    model.add(BatchNormalization(name='normal5'))

    # Passing it to a dense layer
    model.add(Flatten(name='flat1'))

    # 6th layer
    model.add(Dense(4096, 
                    input_shape=(IMG_SIZE*IMG_SIZE*3,), 
                    name='dense1'
                   )
             )
    
    model.add(Activation('relu', name='activ6'))
    model.add(Dropout(0.5, name='drop1')) #to prevent overfitting
    model.add(BatchNormalization())

    # 6th layer
    model.add(Dense(4096, name='dense2'))
    model.add(Activation('relu', name='activ7'))
    model.add(Dropout(0.5, name='drop2')) #to prevent overfitting
    model.add(BatchNormalization())

    # 7th layer
    model.add(Dense(4096, name='dense3'))
    model.add(Activation('relu', name='activ8'))
    model.add(Dropout(0.5, name='drop3'))
    model.add(BatchNormalization())

    # Output Layer
    model.add(Dense(47, name='dense4'))
    model.add(Activation('softmax', name='activ9'))

    #for i, layer in enumerate(model.layers): #To rename all layers
    #    layer.name = 'layer_' + str(i)

    model.summary()

    model.compile(  
        loss='sparse_categorical_crossentropy', # can be 'binary_crossentropy','sparse_categorical_crossentropy','categorical_crossentropy'
        optimizer='sgd', # can be'sgd' or 'adam'
        metrics=['accuracy'],
    )

    model.run_eagerly = False # To allow the use of callbacks tensorboard

    date=str(datetime.now().strftime("%Y%m%d_%H%M%S")) # Generate a unique name based on date and time to save the generated files 
    #savePath=Path(outputPath / date)
    #savePath.mkdir
    #modeldir=Path(MODEL / date)

    logpath=Path(outputPath / date)
    logpath.mkdir()

    tensorboard = keras.callbacks.TensorBoard(log_dir=logpath)
    
    hist=model.fit(trainDataFeatures, trainDataLabel, 
                   callbacks=[tensorboard],
              batch_size=64, # Number of images considered for each epoch
              epochs=500, # Number of runs to train the model
              verbose=1, #Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.
              #validation_split=0.1,
              validation_data=(valDataFeatures,valDataLabel), # Ratio trained/tested data
              shuffle=False, #whether to shuffle the training data before each epoch
             )
    #modeldir.mkdir()
    modelname=os.path.join(date + "_exp2_dtdRotated.model")
    model.save(Path(outputPath / modelname))
    i += 1

# Analyse

In [ ]:
# Parameters to specify

projectDirectory=Path("C:/Users/AQ62270/201910_ccn/NIPS/") # Project path
inputDirectory=Path("2_pipeline/store/") # Location of the model
modelName=Path("20200405_162613_exp2_BS64.model") # Name of the model

outputDirectory=Path("2_pipeline/store/") # Location to save the figures
storeDirectory=Path("2_pipeline/store/") # Location to save the intermediate dataframes
figPath = "C:/Users/AQ62270/201910_ccn/NIPS/2_pipeline/store/" # Location of saved figures for the paper

dataDirectory=Path('C:/Users/AQ62270/201910_ccn/0_data/processed/500-300-0Light_Clean/') # Specify the dataset to evaluate the model

inputPath=Path(projectDirectory/inputDirectory) # Create the full path to the input pictures
outputPath=Path(projectDirectory/outputDirectory) # Create the full path to the output pictures
storePath=Path(projectDirectory/storeDirectory) # Create the full path to the output pictures

intermediateResults=True

# Define design
## Font size
SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 15

plt.rc('font', size=MEDIUM_SIZE)         # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

## Figure dimensions
pageWidth=174/25.4
columnWidth=84/25.4
baseLineNumbering=(0.5,-0.28) # Position of the captions under the sub-figures (letters or numbers)

## Colors
colorValSplit10="#d8b365"
colorValSplit30="#5ab4ac"
markerSplit10="o"
markerSplit30="s"
colorDefect="#f1a340"
colorNodefect="#998ec3"

# Open the model
modelPath=os.path.join(inputPath,modelName)
model = keras.models.load_model(modelPath)
model.summary()
# TO DO - plot the learning curve 
#plt.plot(hist.history['val_accuracy'])

# Get the convolution layers adresses in the model
convLayers=pd.DataFrame({"layerIndex":[], "convolutionIndex":[], "layerName":[]}, dtype=int)
i=0
j=1
for layer in model.layers:
    if "conv" in model.layers[i].name:
        convLayers=convLayers.append({"layerIndex":i, "convolutionIndex":j, "layerName":model.layers[i].name}, ignore_index=True)
        j+=1
    i+=1

layerNumber=[0,4,8,11,14] # Manually select the adress of convolution layers
# TO DO - Do it automatically
# layerNumber=convLayer.loc[].count

# List the categories (If not done previously)
lines = []
buffer=[]
with open(os.path.join(Path('C:/Users/AQ62270/201910_ccn/NIPS/0_data/dtd/labels/test'+DS.iloc[0]["dataset"]+'.txt'))) as f:
    lines = [line.rstrip() for line in f]
    for image in lines:
        buffer.append(os.path.split(image)[0])
        categories = list( dict.fromkeys(buffer) )

if intermediateResults==True:
    print(convLayers)

In [ ]:
# Save maximal value of the convolutional images for each convolution layer

dfFeatureMap = pd.DataFrame({"Layer":[], "filtre" : [], "image":[], "category":[], "maxima":[]}, dtype=int)      

# Create "n" submodels for "n" existing convolution layers
for layer in convLayers["convolutionIndex"]:
    print("Layer number ", layer)
    numberOfFilter = len(model.layers[layerNumber[layer-1]].get_weights()[1])
    modelLayer = Model(inputs=model.inputs, outputs=model.layers[layerNumber[layer-1]].output) # Create the submodel
    
    for category in tqdm(categories):  # For each folder, create a number for each category
        class_num = categories.index(category) 
        for line in lines:
            trueCategory = os.path.split(line)[0]
            if category == trueCategory:
                try:
                    imageName=os.path.split(line)[1]
                    img_array = cv2.imread(os.path.join(Path(dtd,category,os.path.split(line)[1])))  # open the image in grayscale
                    img_array= cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                    img_array = expand_dims(img_array, axis=0)                    
                    for filterNumber in range(numberOfFilter):
                        feature_maps = modelLayer.predict(img_array)
                        dfFeatureMap=dfFeatureMap.append({"Layer": layer,"filtre" : filterNumber, "image":imageName, "category":category, "maxima":np.max(feature_maps[0, :, :, filterNumber])}, ignore_index=True)
                except Exception as e:  # To keep the output clean
                    pass
    dfFeatureMap.to_csv(Path(os.path.join(outputPath,os.path.splitext(modelName)[0]+"_dfFeatureMap.csv")),index=True)
print(dfFeatureMap)
print("Done!")

In [ ]:
dfFeatureMap = pd.read_csv(Path(os.path.join(storePath,os.path.splitext(modelName)[0]+"_dfFeatureMap.csv")))
if len(dfFeatureMap)==(47*(256*2+384*2+96)*40):
    print(dfFeatureMap)
    print("Done!")
else:
    print("Warning : Check the dataframe length")

In [ ]:
# Find the optimized value mutual information threshold for each filter

binsValue=30

dfFeatureMap = pd.read_csv(Path(os.path.join(storePath,os.path.splitext(modelName)[0]+"_dfFeatureMap.csv")))
iconfusionMatrix=pd.DataFrame({"Layer":[], "filter":[],"category":[], "iThreshold":[], "truePositive":[],"trueNegative":[], "falsePositive":[], "falsenegative":[], "mutualScore":[], "iInfoMutuelle":[]}, dtype=int)

for layer in convLayers["convolutionIndex"][1:2]:
    print("Layer number : ", layer)    
    numberOfFilter = len(model.layers[layerNumber[layer-1]].get_weights()[1])
    
    for filterNumber in range (numberOfFilter)[232:]:
        print("filter number : ",filterNumber)
        for category in categories:
            print("category : ",category)
            #dfinodefect=dfFeatureMap[(dfFeatureMap['category']=="nodefect") & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber)]
            #dfidefect=dfFeatureMap[(dfFeatureMap['category']=="defect") & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber)]
            minND=np.amin(dfFeatureMap[(dfFeatureMap['category']==category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber)]["maxima"])
            maxND=np.amax(dfFeatureMap[(dfFeatureMap['category']==category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber)]["maxima"])
            thresholds=np.arange(minND, maxND, (maxND-minND)/20)
            for iThreshold in (thresholds):
                iTP=dfFeatureMap[(dfFeatureMap['category']==category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]>=iThreshold)].count()[4]
                iTN=dfFeatureMap[(dfFeatureMap['category']!=category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]<iThreshold)].count()[4]
                iFP=dfFeatureMap[(dfFeatureMap['category']!=category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]>=iThreshold)].count()[4]
                iFN=dfFeatureMap[(dfFeatureMap['category']==category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]<iThreshold)].count()[4]
                iS=iTP+iTN+iFP+iFN
                label_true=[*np.ones(iTP),*np.zeros(iTN),*np.zeros(iFP),*np.ones(iFN)]
                label_pred=[*np.ones(iTP),*np.zeros(iTN),*np.ones(iFP),*np.zeros(iFN)]
                
                imutualScore=normalized_mutual_info_score(label_true, label_pred)
                iPY1 = (iTP+iFP)/iS
                iPY0 = (iFN+iTN)/iS
                iPC1 = (iTP+iFN)/iS
                iPC0 = (iFP+iTN)/iS
                iPY1C1, iPY0C1, iPY1C0, iPY0C0 = iTP/iS, iFN/iS, iFP/iS, iTN/iS
                
                if (iFP!=0 and iFN!=0 and iTP!=0 and iTN!=0):
                    iHY = - iPY1*math.log2(iPY1) - iPY0*math.log2(iPY0)
                    iHC = - iPC1*math.log2(iPC1) - iPC0*math.log2(iPC0)
                    iHYC = - iPY1C1*math.log2(iPY1C1) - iPY0C1*math.log2(iPY0C1) - iPY1C0*math.log2(iPY1C0) - iPY0C0*math.log2(iPY0C0)
                    iIYC = iHY + iHC - iHYC
                    iconfusionMatrix=iconfusionMatrix.append({"Layer":layer, "filter":filterNumber,"category":category, "iThreshold":iThreshold, "truePositive":iTP,"trueNegative":iTN, "falsePositive":iFP, "falsenegative":iFN, "mutualScore":imutualScore, "iInfoMutuelle":iIYC}, ignore_index=True)        
                else :
                    iconfusionMatrix=iconfusionMatrix.append({"Layer":layer, "filter":filterNumber,"category":category, "iThreshold":iThreshold, "truePositive":iTP,"trueNegative":iTN, "falsePositive":iFP, "falsenegative":iFN, "mutualScore":np.NaN, "iInfoMutuelle":np.NaN}, ignore_index=True)        
                   
        #if intermediateResults==True:
        #    for category in categories[43:44]:
        #        fig, ax1 = plt.subplots()
        #        fig.set_figheight(4)
        #        fig.set_figwidth(4)
        #        ax1.hist(dfFeatureMap[(dfFeatureMap['category']==category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber)]["maxima"], bins=binsValue, alpha=0.8, density=True, label=category)
        #        labelNoCategory=("no "+category)
        #        ax1.hist(dfFeatureMap[(dfFeatureMap['category']!=category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber)]["maxima"], bins=binsValue, alpha=0.8, density=True, label=labelNoCategory)
        #        ax1.legend(loc=2)
        #        ax1.set_ylabel("Frequency")
        #        ax1.set_xlabel("Max. value")
        #        ax1.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
        #        ax2 = ax1.twinx()
        #        ax2.plot(iconfusionMatrix[(iconfusionMatrix["category"]==category)&(iconfusionMatrix["Layer"]==layer)&(iconfusionMatrix["filter"]==filterNumber)]["iThreshold"], iconfusionMatrix[(iconfusionMatrix["category"]==category)&(iconfusionMatrix["Layer"]==layer)&(iconfusionMatrix["filter"]==filterNumber)]["iInfoMutuelle"], '--', label="mutualInfo", color="black")
        #        ax2.set_ylabel("Mutual information")
        #        ax2.legend(loc=1)
        #        plt.title("Filter %i"% filterNumber + " of layer %i" % layer)
        #        plt.plot()
        #        plt.show()
        #        plt.close()

    iconfusionMatrix.to_csv(Path(os.path.join(outputPath,os.path.splitext(modelName)[0]+"_iconfusionMatrix_L2F232_end.csv")),index=True)
print("Done!")

In [ ]:
# Find the optimized value mutual information threshold for each filter

binsValue=30
categories=["spiralled"]
filters=[124,125]
layers=[2]
dfFeatureMap = pd.read_csv(Path(os.path.join(storePath,os.path.splitext(modelName)[0]+"_dfFeatureMap.csv")))
iconfusionMatrix=pd.DataFrame({"Layer":[], "filter":[],"category":[], "iThreshold":[], "truePositive":[],"trueNegative":[], "falsePositive":[], "falsenegative":[], "mutualScore":[], "iInfoMutuelle":[]}, dtype=int)

fig, axes = plt.subplots(1,2,figsize=(pageWidth,3))
for layer in layers:
    print("Layer number : ", layer)    
    numberOfFilter = len(model.layers[layerNumber[layer-1]].get_weights()[1])
    i=0
    for filterNumber in filters:
        print("filter number : ",filterNumber)
#        for category in categories:
#            print("category : ",category)
#            #dfinodefect=dfFeatureMap[(dfFeatureMap['category']=="nodefect") & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber)]
#            #dfidefect=dfFeatureMap[(dfFeatureMap['category']=="defect") & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber)]
#            minND=np.amin(dfFeatureMap[(dfFeatureMap['category']==category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber)]["maxima"])
#            maxND=np.amax(dfFeatureMap[(dfFeatureMap['category']==category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber)]["maxima"])
#            thresholds=np.arange(minND, maxND, (maxND-minND)/20)
#            for iThreshold in (thresholds):
#                iTP=dfFeatureMap[(dfFeatureMap['category']==category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]>=iThreshold)].count()[4]
#                iTN=dfFeatureMap[(dfFeatureMap['category']!=category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]<iThreshold)].count()[4]
#                iFP=dfFeatureMap[(dfFeatureMap['category']!=category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]>=iThreshold)].count()[4]
#                iFN=dfFeatureMap[(dfFeatureMap['category']==category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]<iThreshold)].count()[4]
#                iS=iTP+iTN+iFP+iFN
#                label_true=[*np.ones(iTP),*np.zeros(iTN),*np.zeros(iFP),*np.ones(iFN)]
#                label_pred=[*np.ones(iTP),*np.zeros(iTN),*np.ones(iFP),*np.zeros(iFN)]
#                
#                imutualScore=normalized_mutual_info_score(label_true, label_pred)
#                iPY1 = (iTP+iFP)/iS
#                iPY0 = (iFN+iTN)/iS
#                iPC1 = (iTP+iFN)/iS
#                iPC0 = (iFP+iTN)/iS
#                iPY1C1, iPY0C1, iPY1C0, iPY0C0 = iTP/iS, iFN/iS, iFP/iS, iTN/iS
#                
#                if (iFP!=0 and iFN!=0 and iTP!=0 and iTN!=0):
#                    iHY = - iPY1*math.log2(iPY1) - iPY0*math.log2(iPY0)
#                    iHC = - iPC1*math.log2(iPC1) - iPC0*math.log2(iPC0)
#                    iHYC = - iPY1C1*math.log2(iPY1C1) - iPY0C1*math.log2(iPY0C1) - iPY1C0*math.log2(iPY1C0) - iPY0C0*math.log2(iPY0C0)
#                    iIYC = iHY + iHC - iHYC
#                    iconfusionMatrix=iconfusionMatrix.append({"Layer":layer, "filter":filterNumber,"category":category, "iThreshold":iThreshold, "truePositive":iTP,"trueNegative":iTN, "falsePositive":iFP, "falsenegative":iFN, "mutualScore":imutualScore, "iInfoMutuelle":iIYC}, ignore_index=True)        
#                else :
#                    iconfusionMatrix=iconfusionMatrix.append({"Layer":layer, "filter":filterNumber,"category":category, "iThreshold":iThreshold, "truePositive":iTP,"trueNegative":iTN, "falsePositive":iFP, "falsenegative":iFN, "mutualScore":np.NaN, "iInfoMutuelle":np.NaN}, ignore_index=True)        
#                   
        for category in categories:
            fig, ax1 = plt.subplots()
            fig.set_figheight(4)
            fig.set_figwidth(4)
            axes[i].hist(dfFeatureMap[(dfFeatureMap['category']==category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber)]["maxima"],color=colorDefect, bins=binsValue, alpha=0.8, density=True, label=category)
            labelNoCategory=("no "+category)
            axes[i].hist(dfFeatureMap[(dfFeatureMap['category']!=category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber)]["maxima"],color=colorNodefect, bins=binsValue, alpha=0.8, density=True, label=labelNoCategory)
            axes[i].legend(loc=2)
            axes[i].set_ylabel("Frequency")
            axes[i].set_xlabel("Max. value")
            axes[i].yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
            ax2=axes[i].twinx()
            ax2.set_xticks(np.arrange(0,0.4,step=0.01))
            ax2.plot(iconfusionMatrix[(iconfusionMatrix["category"]==category)&(iconfusionMatrix["Layer"]==layer)&(iconfusionMatrix["filter"]==filterNumber)]["iThreshold"], iconfusionMatrix[(iconfusionMatrix["category"]==category)&(iconfusionMatrix["Layer"]==layer)&(iconfusionMatrix["filter"]==filterNumber)]["iInfoMutuelle"], '--', label="mutualInfo", color="black")
            ax2.set_ylabel("Mutual information")
            #ax2.legend(loc=1)          
            axes[i].set_title("Filter %i"% filterNumber + " of layer %i" % layer)
        i+=1
    plt.tight_layout()
    plt.plot()
    plt.show()
    plt.close()

    #iconfusionMatrix.to_csv(Path(os.path.join(outputPath,os.path.splitext(modelName)[0]+"_iconfusionMatrix_L2F232_end.csv")),index=True)
print("Done!")

In [ ]:
iconfusionMatrix.to_csv(Path(os.path.join(outputPath,os.path.splitext(modelName)[0]+"_iconfusionMatrix_L2F134_231.csv")),index=True)


In [ ]:
# Find the optimized value mutual information threshold for each filter

binsValue=30

dfFeatureMap = pd.read_csv(Path(os.path.join(storePath,os.path.splitext(modelName)[0]+"_dfFeatureMap.csv")))
iconfusionMatrix=pd.DataFrame({"Layer":[], "filter":[],"category":[], "iThreshold":[], "truePositive":[],"trueNegative":[], "falsePositive":[], "falsenegative":[], "mutualScore":[], "iInfoMutuelle":[]}, dtype=int)

for layer in convLayers["convolutionIndex"][3:4]:
    print("Layer number : ", layer)    
    numberOfFilter = len(model.layers[layerNumber[layer-1]].get_weights()[1])
    
    for filterNumber in range (numberOfFilter)[333:]:
        print("filter number : ",filterNumber)
        for category in categories:
            print("category : ",category)
            #dfinodefect=dfFeatureMap[(dfFeatureMap['category']=="nodefect") & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber)]
            #dfidefect=dfFeatureMap[(dfFeatureMap['category']=="defect") & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber)]
            minND=np.amin(dfFeatureMap[(dfFeatureMap['category']==category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber)]["maxima"])
            maxND=np.amax(dfFeatureMap[(dfFeatureMap['category']==category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber)]["maxima"])
            thresholds=np.arange(minND, maxND, (maxND-minND)/20)
            for iThreshold in (thresholds):
                iTP=dfFeatureMap[(dfFeatureMap['category']==category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]>=iThreshold)].count()[4]
                iTN=dfFeatureMap[(dfFeatureMap['category']!=category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]<iThreshold)].count()[4]
                iFP=dfFeatureMap[(dfFeatureMap['category']!=category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]>=iThreshold)].count()[4]
                iFN=dfFeatureMap[(dfFeatureMap['category']==category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]<iThreshold)].count()[4]
                iS=iTP+iTN+iFP+iFN
                label_true=[*np.ones(iTP),*np.zeros(iTN),*np.zeros(iFP),*np.ones(iFN)]
                label_pred=[*np.ones(iTP),*np.zeros(iTN),*np.ones(iFP),*np.zeros(iFN)]
                
                imutualScore=normalized_mutual_info_score(label_true, label_pred)
                iPY1 = (iTP+iFP)/iS
                iPY0 = (iFN+iTN)/iS
                iPC1 = (iTP+iFN)/iS
                iPC0 = (iFP+iTN)/iS
                iPY1C1, iPY0C1, iPY1C0, iPY0C0 = iTP/iS, iFN/iS, iFP/iS, iTN/iS
                
                if (iFP!=0 and iFN!=0 and iTP!=0 and iTN!=0):
                    iHY = - iPY1*math.log2(iPY1) - iPY0*math.log2(iPY0)
                    iHC = - iPC1*math.log2(iPC1) - iPC0*math.log2(iPC0)
                    iHYC = - iPY1C1*math.log2(iPY1C1) - iPY0C1*math.log2(iPY0C1) - iPY1C0*math.log2(iPY1C0) - iPY0C0*math.log2(iPY0C0)
                    iIYC = iHY + iHC - iHYC
                    iconfusionMatrix=iconfusionMatrix.append({"Layer":layer, "filter":filterNumber,"category":category, "iThreshold":iThreshold, "truePositive":iTP,"trueNegative":iTN, "falsePositive":iFP, "falsenegative":iFN, "mutualScore":imutualScore, "iInfoMutuelle":iIYC}, ignore_index=True)        
                else :
                    iconfusionMatrix=iconfusionMatrix.append({"Layer":layer, "filter":filterNumber,"category":category, "iThreshold":iThreshold, "truePositive":iTP,"trueNegative":iTN, "falsePositive":iFP, "falsenegative":iFN, "mutualScore":np.NaN, "iInfoMutuelle":np.NaN}, ignore_index=True)        
                   
        #if intermediateResults==True:
        #    for category in categories[43:44]:
        #        fig, ax1 = plt.subplots()
        #        fig.set_figheight(4)
        #        fig.set_figwidth(4)
        #        ax1.hist(dfFeatureMap[(dfFeatureMap['category']==category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber)]["maxima"], bins=binsValue, alpha=0.8, density=True, label=category)
        #        labelNoCategory=("no "+category)
        #        ax1.hist(dfFeatureMap[(dfFeatureMap['category']!=category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber)]["maxima"], bins=binsValue, alpha=0.8, density=True, label=labelNoCategory)
        #        ax1.legend(loc=2)
        #        ax1.set_ylabel("Frequency")
        #        ax1.set_xlabel("Max. value")
        #        ax1.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
        #        ax2 = ax1.twinx()
        #        ax2.plot(iconfusionMatrix[(iconfusionMatrix["category"]==category)&(iconfusionMatrix["Layer"]==layer)&(iconfusionMatrix["filter"]==filterNumber)]["iThreshold"], iconfusionMatrix[(iconfusionMatrix["category"]==category)&(iconfusionMatrix["Layer"]==layer)&(iconfusionMatrix["filter"]==filterNumber)]["iInfoMutuelle"], '--', label="mutualInfo", color="black")
        #        ax2.set_ylabel("Mutual information")
        #        ax2.legend(loc=1)
        #        plt.title("Filter %i"% filterNumber + " of layer %i" % layer)
        #        plt.plot()
        #        plt.show()
        #        plt.close()

    iconfusionMatrix.to_csv(Path(os.path.join(outputPath,os.path.splitext(modelName)[0]+"_iconfusionMatrix_L4F108_end.csv")),index=True)
print("Done!")

In [ ]:
# Built the confusion matrix with optimized value of mutual information for each filter

iconfusionMatrix = pd.read_csv(Path(os.path.join(storePath,os.path.splitext(modelName)[0]+"_iconfusionMatrix.txt")))
confusionMatrix=pd.DataFrame({"Layer":[], "filter":[], "category":[], "threshold":[], "truePositive":[],"trueNegative":[], "falsePositive":[], "falsenegative":[], "mutualScore":[], "mutualInfo":[]}, dtype=int)

for layer in convLayers["convolutionIndex"][2:3]:
    numberOfFilter = len(model.layers[layerNumber[layer-1]].get_weights()[1])
    print(layer)
    for filterNumber in range (133,384):
        print(filterNumber)
        for category in categories:
            threshold=iconfusionMatrix.loc[(iconfusionMatrix["category"]==category)&(iconfusionMatrix["Layer"]==layer)&(iconfusionMatrix["filter"]==filterNumber)].sort_values(by="iInfoMutuelle", ascending=False).iloc[0]["iThreshold"]
            
            TP=dfFeatureMap[(dfFeatureMap['category']==category) &  (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]>=threshold)].count()[4]
            TN=dfFeatureMap[(dfFeatureMap['category']!=category) &  (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]<threshold)].count()[4]
            FP=dfFeatureMap[(dfFeatureMap['category']!=category) &  (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]>=threshold)].count()[4]
            FN=dfFeatureMap[(dfFeatureMap['category']==category) &  (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]<threshold)].count()[4]
            S=TP+TN+FP+FN
            
            # Calcul of the mutual information
            PY1 = (TP+FP)/S
            PY0 = (FN+TN)/S
            PC1 = (TP+FN)/S
            PC0 = (FP+TN)/S 
            PY1C1, PY0C1, PY1C0, PY0C0 = TP/S, FN/S, FP/S, TN/S
            
            HY = - PY1*math.log2(PY1) - PY0*math.log2(PY0)
            HC = - PC1*math.log2(PC1) - PC0*math.log2(PC0)
            HYC = - PY1C1*math.log2(PY1C1) - PY0C1*math.log2(PY0C1) - PY1C0*math.log2(PY1C0) - PY0C0*math.log2(PY0C0)     
            IYC = HY + HC - HYC
        
            label_true=[*np.ones(TP),*np.zeros(TN),*np.zeros(FP),*np.ones(FN)]
            label_pred=[*np.ones(TP),*np.zeros(TN),*np.ones(FP),*np.zeros(FN)]
            mutualScore=normalized_mutual_info_score(label_true, label_pred)
    
            confusionMatrix=confusionMatrix.append({"Layer":layer, "filter":filterNumber,"category":category, "threshold":threshold, "truePositive":TP,"trueNegative":TN, "falsePositive":FP, "falsenegative":FN, "mutualScore":mutualScore, "mutualInfo":IYC}, ignore_index=True)

confusionMatrix.to_csv(Path(os.path.join(storePath,os.path.splitext(modelName)[0]+"_confusionMatrixtest.csv")),index=True)


In [ ]:
# Plot entropy histogram
confusionMatrix = pd.read_csv(Path(os.path.join(storePath,os.path.splitext(modelName)[0]+"_confusionMatrix.csv")))

viridis = cm.get_cmap('viridis', 12)

fig, axes = plt.subplots(3,2,figsize=(pageWidth,7))

i=0.1
j=0
for category in categories[:3]:
    tab=pd.DataFrame({"layer":[], "decile":[]})

    for layer in convLayers["convolutionIndex"]:
    
        axes[j,0].hist(confusionMatrix[(confusionMatrix["Layer"]==layer)&(confusionMatrix["category"]==category)].sort_values(by="mutualInfo", ascending=False)["mutualInfo"], bins=30, alpha=0.7, color=viridis(i), density=True, label={"Layer %i" %layer})
        tab=tab.append({"layer":layer, "decile":np.percentile(confusionMatrix[(confusionMatrix["Layer"]==layer)&(confusionMatrix["category"]==category)]["mutualInfo"],np.arange(0,100,10))[9]},ignore_index=True)
        axes[j,1].scatter(tab["layer"][layer-1],tab["decile"][layer-1],color=viridis(i),s=70, label="Layer %i" %layer)
        i+=0.3
        
        axes[j,0].set_ylabel("Frequency")
        axes[j,0].set_xlabel("Mutual information")
        axes[j,0].legend()
        #axes[j,0].text(baseLineNumbering[0],baseLineNumbering[1],"(a)", horizontalalignment='center',verticalalignment='center', transform=axes[0].transAxes)

        axes[j,1].yaxis.set_label_position("right")
        axes[j,1].yaxis.tick_right()
        #axes[j,1].text(baseLineNumbering[0],baseLineNumbering[1],"(b)", horizontalalignment='center',verticalalignment='center', transform=axes[1].transAxes)
        axes[j,1].legend()
        axes[j,1].set_xlabel("Layer")
        axes[j,1].set_xlim(0,6)
        axes[j,1].set_xticks(np.arange(1, 6, step=1))
        axes[j,1].set_ylabel("Mutual information \n of last decile")
    j+=1
plt.tight_layout()
plt.savefig(Path(os.path.join(figPath,os.path.splitext(modelName)[0]+"_mutualInfoLayers.pdf")))
plt.show()
plt.close()

In [ ]:
# Plot mutual information (last decile) through the Newkork (sorted) - Pandas version - DRAFT

#confusionMatrix = pd.read_csv(Path(os.path.join(storePath,os.path.splitext(modelName)[0]+"_confusionMatrix.csv")))

viridis = cm.get_cmap('viridis', 12)
fig, ax = plt.subplots(1,1,figsize=(pageWidth*2,1))
i=0.1
layers = [1,2,3,4]
tab=pd.DataFrame({"layer":[],"category":[], "decile":[], "category":[]})

for layer in layers:
    for category in categories:    
        tab=tab.append({"layer":layer, "category":category, "decile":np.percentile(confusionMatrix[(confusionMatrix["Layer"]==layer)&(confusionMatrix["category"]==category)]["mutualInfo"],np.arange(0,100,10))[9], "category":category},ignore_index=True)

x=[]
for layer in (layers):
    x1=tab[tab["layer"]==layer].sort_values(by="category", ascending=True)["decile"]
    x=np.append(x,x1,axis=0)
x=x.reshape(4,47)
g=x[:,np.argsort(x[0])]
im = ax.imshow(x)
ax.set_yticks(np.arange(0,len(layers)))
ax.set_yticklabels(layers)
ax.set_xticks(np.arange(len(categories)))
ax.set_xticklabels(tab[tab["layer"]==layer].sort_values(by="category", ascending=True)["category"])
ax.set_ylabel('Layers')
ax.set_title("Mutual information score through the network (last decile)")
ax.vlines(np.arange(-0.5,len(categories)-1), -.5, len(layers)-.5, 'w')
plt.setp(ax.get_xticklabels(), rotation=90, va="center", ha="right",
         rotation_mode="anchor")
cbar=fig.colorbar(im,ax=ax, pad=0.01, aspect=5, ticks=[0,0.01,0.02])
cbar.set_ticks([0,0.01,0.02])
cbar.set_ticklabels([0,0.01,0.02])

plt.show()

In [ ]:
# Plot entropy all the histograms by category 

confusionMatrix = pd.read_csv(Path(os.path.join(storePath,os.path.splitext(modelName)[0]+"_confusionMatrix.csv")))
layers = [1,2,3,4,5]
for nbOfGraph in range (0,10):
    viridis = cm.get_cmap('viridis', 5)
    
    fig, axes = plt.subplots(2,6,figsize=(pageWidth,3))
    
    tab=pd.DataFrame({"layer":[], "decile":[], "category":[]})
    subcategories=categories[5*nbOfGraph:5+5*nbOfGraph]
    
    layerLegend=[]
    for layer in layers:
        layerLegend.append(Line2D([0], [0], color=viridis(layers.index(layer)/len(layers)), lw=4))
        for category in subcategories:
            ax_category=axes[0,subcategories.index(category)] # Specify the subplot object
            sns.distplot(confusionMatrix[(confusionMatrix["Layer"]==layer)&(confusionMatrix["category"]==category)].sort_values(by="mutualInfo", ascending=False)["mutualInfo"], 
                         ax=ax_category, 
                         color=viridis(layers.index(layer)/len(layers)),
                         bins=30, 
                         hist=False,
                        )
            sns.despine(ax=ax_category,top=True, right=True, left=True, bottom=False)
            ax_category.set_xlabel("%s" %category)
            ax_category.xaxis.set_ticks_position('bottom')
            ax_category.xaxis.set_label_position('top')
            #axes0,[0].text(baseLineNumbering[0],baseLineNumbering[1],"(a)", horizontalalignment='center',verticalalignment='center', transform=axes[0].transAxes)
        axes[0,0].set_ylabel("Frequency")
        axes[0,5].axis('off')
        axes[0,5].legend(layerLegend,layers,frameon=False)
    categoryLegend=[]
    for category in subcategories:
        categoryLegend.append(Line2D([0], [0], color=sns.color_palette("Set1", n_colors=len(layers))[subcategories.index(category)], lw=4))
        for layer in layers:
            ax_layer=axes[1,layers.index(layer)]
            sns.distplot(confusionMatrix[(confusionMatrix["Layer"]==layer)&(confusionMatrix["category"]==category)].sort_values(by="mutualInfo", ascending=False)["mutualInfo"], 
                         ax=ax_layer, 
                         bins=30,
                         hist=False,
                         color=sns.color_palette("Set1", n_colors=len(layers))[subcategories.index(category)],
                        )
            
            sns.despine(ax=ax_layer,top=True, right=True, left=True, bottom=False)
            axes[1,0].set_ylabel("Frequency")
            ax_layer.set_xlabel("Layer %i" %layer)
            #axes[j,0].text(baseLineNumbering[0],baseLineNumbering[1],"(a)", horizontalalignment='center',verticalalignment='center', transform=axes[0].transAxes)
        axes[1,5].axis('off')
        shortlist=[]
        for short in subcategories:
            shortlist.append(short[:4])
        axes[1,5].legend(categoryLegend,shortlist,frameon=False)
    
    plt.tight_layout()
    plt.show()
    plt.savefig(Path(os.path.join(figPath,os.path.splitext(modelName)[0]+"_mutualInfoLayers.pdf")))
    plt.close()

In [ ]:
# Plot entropy the selected histograms by category 

confusionMatrix = pd.read_csv(Path(os.path.join(storePath,os.path.splitext(modelName)[0]+"_confusionMatrix.csv")))
layers = [1,2,3,4,5]
viridis = cm.get_cmap('viridis', 5)

fig, axes = plt.subplots(2,6,figsize=(pageWidth,3))

tab=pd.DataFrame({"layer":[], "decile":[], "category":[]})
subcategories=["cobwebbed","knitted","marbled","spiralled","studded"]

layerLegend=[]
for layer in layers:
    layerLegend.append(Line2D([0], [0], color=viridis(layers.index(layer)/len(layers)), lw=4))
    for category in subcategories:
        ax_category=axes[0,subcategories.index(category)] # Specify the subplot object
        sns.distplot(confusionMatrix[(confusionMatrix["Layer"]==layer)&(confusionMatrix["category"]==category)].sort_values(by="mutualInfo", ascending=False)["mutualInfo"], 
                     ax=ax_category, 
                     color=viridis(layers.index(layer)/len(layers)),
                     bins=30, 
                     hist=False,
                    )
        sns.despine(ax=ax_category,top=True, right=True, left=True, bottom=False)
        ax_category.set_xlabel("%s" %category)
        ax_category.xaxis.set_ticks_position('bottom')
        ax_category.xaxis.set_label_position('top')
        #axes0,[0].text(baseLineNumbering[0],baseLineNumbering[1],"(a)", horizontalalignment='center',verticalalignment='center', transform=axes[0].transAxes)
    axes[0,0].set_ylabel("Frequency")
    axes[0,5].axis('off')
    axes[0,5].legend(layerLegend,layers,frameon=False)
categoryLegend=[]
for category in subcategories:
    categoryLegend.append(Line2D([0], [0], color=sns.color_palette("Set1", n_colors=len(layers))[subcategories.index(category)], lw=4))
    for layer in layers:
        ax_layer=axes[1,layers.index(layer)]
        sns.distplot(confusionMatrix[(confusionMatrix["Layer"]==layer)&(confusionMatrix["category"]==category)].sort_values(by="mutualInfo", ascending=False)["mutualInfo"], 
                     ax=ax_layer, 
                     bins=30,
                     hist=False,
                     color=sns.color_palette("Set1", n_colors=len(layers))[subcategories.index(category)],
                    )
        
        sns.despine(ax=ax_layer,top=True, right=True, left=True, bottom=False)
        axes[1,0].set_ylabel("Frequency")
        ax_layer.set_xlabel("Layer %i" %layer)
        #axes[j,0].text(baseLineNumbering[0],baseLineNumbering[1],"(a)", horizontalalignment='center',verticalalignment='center', transform=axes[0].transAxes)
    axes[1,5].axis('off')
    shortlist=[]
    for short in subcategories:
        shortlist.append(short[:4])
    axes[1,5].legend(categoryLegend,shortlist,frameon=False)

plt.tight_layout()
plt.show()
plt.savefig(Path(os.path.join(figPath,os.path.splitext(modelName)[0]+"_mutualInfoLayersSelected.pdf")))
plt.close()

In [ ]:
# Plot mutual information (last decile) through the Newkork (sorted) - Pandas version - DRAFT

#confusionMatrix = pd.read_csv(Path(os.path.join(storePath,os.path.splitext(modelName)[0]+"_confusionMatrix.csv")))

viridis = cm.get_cmap('viridis', 12)
fig, ax = plt.subplots(1,1,figsize=(pageWidth*2,1))
i=0.1
layers = [1,2,3,4,5]
tab=pd.DataFrame({"layer":[],"category":[], "decile":[], "category":[]})

for layer in layers:
    for category in categories:    
        tab=tab.append({"layer":layer, "category":category, "decile":np.percentile(confusionMatrix[(confusionMatrix["Layer"]==layer)&(confusionMatrix["category"]==category)]["mutualInfo"],np.arange(0,100,10))[9], "category":category},ignore_index=True)

x=[]
for layer in (layers):
    x1=tab[tab["layer"]==layer].sort_values(by="category", ascending=True)["decile"]
    x=np.append(x,x1,axis=0)
x=x.reshape(5,47)
g=x[:,np.argsort(x[0])]
im = ax.imshow(x)
ax.set_yticks(np.arange(0,len(layers)))
ax.set_yticklabels(layers)
ax.set_xticks(np.arange(len(categories)))
ax.set_xticklabels(tab[tab["layer"]==layer].sort_values(by="category", ascending=True)["category"])
ax.set_ylabel('Layers')
ax.set_title("Mutual information score through the network (last decile)")
ax.vlines(np.arange(-0.5,len(categories)-1), -.5, len(layers)-.5, 'w')
plt.setp(ax.get_xticklabels(), rotation=90, va="center", ha="right",
         rotation_mode="anchor")
cbar=fig.colorbar(im,ax=ax, pad=0.01, aspect=5, ticks=[0,0.01,0.02])
cbar.set_ticks([0,0.01,0.02])
cbar.set_ticklabels([0,0.01,0.02])

plt.show()
plt.savefig(Path(os.path.join(figPath,os.path.splitext(modelName)[0]+"_LastDecileMap.pdf")))
plt.close()

In [ ]:
# Test
from PIL import Image 
import sys

layer=0
numberOfClasses=5
imagesPerCategory=3
IMG_SIZE=350
numberOfColumns=1+imagesPerCategory

fig, axes = plt.subplots(numberOfColumns,2,figsize=(pageWidth,5))
fig.subplots_adjust(hspace=0)

tab=pd.DataFrame({"category":[],
                  "decile":[]
                 })
# Highest values
for category in sortedCategory[::-1]:
    decile=np.percentile(confusionMatrix[(confusionMatrix["Layer"]==layer+1)&(confusionMatrix["category"]==category)]["mutualInfo"],np.arange(0,100,10))[9]
    tab=tab.append({"category":category, 
                    "decile":decile,
                   },
                   ignore_index=True)
    tab=tab.sort_values(by="decile", ascending=False)

axes[0,0].bar(tab[:numberOfClasses]["category"],
                tab[:numberOfClasses]["decile"])
#axes[0,0].axis('off')
axes[0,0].set_yticks([0,0.01,0.02])
axes[0,0].set_ylabel("Mutual \n information")
ax.set_yticklabels(layers)
#axes[0,0].get_shared_y_axes().join(axes[0,0], axes[0,1])
plt.setp(axes[0,0].get_xticklabels(), 
         rotation=45, 
         va="center", 
         ha="right",
         rotation_mode="anchor")

i=1
for imageNumber in np.arange(0,imagesPerCategory,step=1):
    listImages=[]
    #sortedCategory=y[0,np.argsort(x[layer])] #test
    #print(sortedCategory)
    for category in sortedCategory[::-1][:numberOfClasses]:
     #for category in sortedCategory[::][:numberOfClasses]:
        #print(category)
        imgName=os.listdir("/home/quentin/Documents/NIPS/0_data/dtd/images/"+category+"/")[imageNumber]
        imgPath = ("/home/quentin/Documents/NIPS/0_data/dtd/images/"+category+"/"+imgName) 
        listImages.append(imgPath)
    images = [Image.open(x) for x in listImages]
    #widths, heights = zip(*(i.size for i in images))
    total_width = IMG_SIZE*numberOfClasses
    max_height = IMG_SIZE
    new_im = Image.new('RGB', (total_width, max_height))
    x_offset = 0
    for im in images:
        im = im.resize((IMG_SIZE,IMG_SIZE))
        new_im.paste(im, (x_offset,0))
        x_offset += im.size[0]
    axes[i,0].imshow(new_im)
    axes[i,0].axis('off')
    i+=1
    
#Lowest values
plt.setp(axes[0,1].get_xticklabels(), 
         rotation=45, 
         va="center", 
         ha="right",
         rotation_mode="anchor")

axes[0,1].bar(tab[-numberOfClasses::]["category"],
                tab[-numberOfClasses::]["decile"])
#axes[0,1].axis('off')
axes[0,1].set_yticks([0,0.01,0.02])
i=1
for imageNumber in np.arange(0,imagesPerCategory,step=1):
    listImages=[]
    for category in sortedCategory[numberOfClasses*2-1::-1][numberOfClasses:]: # To rewrite
     #for category in sortedCategory[::][:numberOfClasses]:
        imgName=os.listdir("/home/quentin/Documents/NIPS/0_data/dtd/images/"+category+"/")[imageNumber]
        imgPath = ("/home/quentin/Documents/NIPS/0_data/dtd/images/"+category+"/"+imgName) 
        listImages.append(imgPath)
    images = [Image.open(x) for x in listImages]
    #widths, heights = zip(*(i.size for i in images))
    total_width = IMG_SIZE*numberOfClasses
    max_height = IMG_SIZE
    new_im2 = Image.new('RGB', (total_width, max_height))
    x_offset = 0
    for im in images:
        im = im.resize((IMG_SIZE,IMG_SIZE))
        new_im2.paste(im, (x_offset,0))
        x_offset += im.size[0]
    axes[i,1].imshow(new_im2)
    axes[i,1].axis('off')
    i+=1
plt.tight_layout()
plt.savefig("/home/quentin/Documents/NIPS/2_pipeline/store/fig_example.pdf")
plt.show()
plt.close()


# train new model

In [ ]:
## Parameters to specify
projectDirectory=Path("C:/Users/AQ62270/201910_ccn/NIPS/") # Project path
inputDirectory=Path("2_pipeline/store/") # Location of folders with pictures (in subfolders "nodefect" & "defect")
inputPath=Path(projectDirectory/inputDirectory) # Create the full path to the input pictures
#MODEL=Path(projectDirectory / "mod/") # Where to save the dataset
#LOG=Path(projectDirectory / "log/")
IMG_SIZE = 227 # has to fit with the value in "6_serialize_dataset"
outputDirectory=Path("2_pipeline/tmp/") # Location to save the models and logs
outputPath=Path(projectDirectory/outputDirectory) # Create the full path to the output pictures

# Define the design of experiments
DOE = pd.DataFrame({"dataset" : []}) # Create a dataframe to run the script in batch mode
#DOE = DOE.append({"dataset" : "1"},ignore_index=True)
DOE = DOE.append({"dataset" : "2"},ignore_index=True)
#DOE = DOE.append({"dataset" : "3"},ignore_index=True)
#DOE = DOE.append({"dataset" : "4"},ignore_index=True)
#DOE = DOE.append({"dataset" : "5"},ignore_index=True)
#DOE = DOE.append({"dataset" : "6"},ignore_index=True)
#DOE = DOE.append({"dataset" : "7"},ignore_index=True)
#DOE = DOE.append({"dataset" : "8"},ignore_index=True)
#DOE = DOE.append({"dataset" : "9"},ignore_index=True)
#DOE = DOE.append({"dataset" : "10"},ignore_index=True)

## Training the model
i = 0
while i < DOE.count()[0]:
    # File and folder locations
    trainDataLabel = pickle.load(open(Path(inputPath / os.path.join(DOE.iloc[i]["dataset"][0] + "_traindatalabel.pickle")),"rb"))
    valDataLabel = pickle.load(open(Path(inputPath / os.path.join(DOE.iloc[i]["dataset"][0] + "_valdatalabel.pickle")),"rb"))
    
    trainDataFeatures = pickle.load(open(Path(inputPath / os.path.join(DOE.iloc[i]["dataset"][0] + "_traindatafeatures.pickle")),"rb"))
    valDataFeatures = pickle.load(open(Path(inputPath / os.path.join(DOE.iloc[i]["dataset"][0] + "_valdatafeatures.pickle")),"rb"))

    ## Build the model: 
    model = Sequential()

    # 1st layer
    model.add(Conv2D(filters=96, 
                     input_shape=(IMG_SIZE,IMG_SIZE,3), 
                     kernel_size=(11,11), 
                     strides=(4,4), 
                     padding="valid", 
                     data_format="channels_last", 
                     name='conv1'
                    )
             )
    model.add(Activation('relu', name='activ1'))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding="valid", data_format="channels_last", name='pool1'))
    model.add(BatchNormalization(name='normal1'))

    # 2nd layer
    model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding="valid", data_format="channels_last", name='conv2'))
    model.add(Activation('relu', name='activ2'))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding="valid", data_format="channels_last", name='pool2'))
    model.add(BatchNormalization(name='normal2'))

    # 3rd layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="valid", data_format="channels_last", name='conv3'))
    model.add(Activation('relu', name='activ3'))
    model.add(BatchNormalization(name='normal3'))

    # Passing it to a dense layer
    model.add(Flatten(name='flat1'))

    # 6th layer
    model.add(Dense(4096, 
                    input_shape=(IMG_SIZE*IMG_SIZE*3,), 
                    name='dense1'
                   )
             )
    
    model.add(Activation('relu', name='activ6'))
    model.add(Dropout(0.5, name='drop1')) #to prevent overfitting
    model.add(BatchNormalization())

    # 6th layer
    model.add(Dense(4096, name='dense2'))
    model.add(Activation('relu', name='activ7'))
    model.add(Dropout(0.5, name='drop2')) #to prevent overfitting
    model.add(BatchNormalization())

    # 7th layer
    model.add(Dense(4096, name='dense3'))
    model.add(Activation('relu', name='activ8'))
    model.add(Dropout(0.5, name='drop3'))
    model.add(BatchNormalization())

    # Output Layer
    model.add(Dense(47, name='dense4'))
    model.add(Activation('softmax', name='activ9'))

    #for i, layer in enumerate(model.layers): #To rename all layers
    #    layer.name = 'layer_' + str(i)

    model.summary()

    model.compile(  
        loss='sparse_categorical_crossentropy', # can be 'binary_crossentropy','sparse_categorical_crossentropy','categorical_crossentropy'
        optimizer='sgd', # can be'sgd' or 'adam'
        metrics=['accuracy'],
    )

    model.run_eagerly = False # To allow the use of callbacks tensorboard

    date=str(datetime.now().strftime("%Y%m%d_%H%M%S")) # Generate a unique name based on date and time to save the generated files 
    #savePath=Path(outputPath / date)
    #savePath.mkdir
    #modeldir=Path(MODEL / date)

    logpath=Path(outputPath / date)
    logpath.mkdir()

    tensorboard = keras.callbacks.TensorBoard(log_dir=logpath)
    
    hist=model.fit(trainDataFeatures, trainDataLabel, 
                   callbacks=[tensorboard],
              batch_size=64, # Number of images considered for each epoch
              epochs=500, # Number of runs to train the model
              verbose=1, #Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.
              #validation_split=0.1,
              validation_data=(valDataFeatures,valDataLabel), # Ratio trained/tested data
              shuffle=False, #whether to shuffle the training data before each epoch
             )
    #modeldir.mkdir()
    modelname=os.path.join(date + "_exp2_BS64_3layers.model")
    model.save(Path(outputPath / modelname))
    i += 1

In [ ]:
# Parameters to specify

projectDirectory=Path("C:/Users/AQ62270/201910_ccn/NIPS/") # Project path
inputDirectory=Path("2_pipeline/store/") # Location of the model
modelName=Path("20200507_004956_exp2_BS64_3layers.model") # Name of the model

outputDirectory=Path("2_pipeline/store/") # Location to save the figures
storeDirectory=Path("2_pipeline/store/") # Location to save the intermediate dataframes
figPath = "C:/Users/AQ62270/201910_ccn/NIPS/2_pipeline/store/" # Location of saved figures for the paper

inputPath=Path(projectDirectory/inputDirectory) # Create the full path to the input pictures
outputPath=Path(projectDirectory/outputDirectory) # Create the full path to the output pictures
storePath=Path(projectDirectory/storeDirectory) # Create the full path to the output pictures

intermediateResults=True

# Define design
## Font size
SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 15

plt.rc('font', size=MEDIUM_SIZE)         # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

## Figure dimensions
pageWidth=174/25.4
columnWidth=84/25.4
baseLineNumbering=(0.5,-0.28) # Position of the captions under the sub-figures (letters or numbers)

## Colors
colorValSplit10="#d8b365"
colorValSplit30="#5ab4ac"
markerSplit10="o"
markerSplit30="s"
colorDefect="#f1a340"
colorNodefect="#998ec3"
# Open the model
modelPath=os.path.join(inputPath,modelName)
model = keras.models.load_model("C:/Users/AQ62270/201910_ccn/NIPS/2_pipeline/store/20200507_004956_exp2_BS64_3layers.model")
model.summary()
# TO DO - plot the learning curve 
#plt.plot(hist.history['val_accuracy'])

# Get the convolution layers adresses in the model
convLayers=pd.DataFrame({"layerIndex":[], "convolutionIndex":[], "layerName":[]}, dtype=int)
i=0
j=1
for layer in model.layers:
    if "conv" in model.layers[i].name:
        convLayers=convLayers.append({"layerIndex":i, "convolutionIndex":j, "layerName":model.layers[i].name}, ignore_index=True)
        j+=1
    i+=1

#layerNumber=[0,4,8,11,14] # Manually select the adress of convolution layers
# TO DO - Do it automatically
# layerNumber=convLayer.loc[].count

# List the categories (If not done previously)
lines = []
buffer=[]
with open(os.path.join(Path('C:/Users/AQ62270/201910_ccn/NIPS/0_data/dtd/labels/test'+DS.iloc[0]["dataset"]+'.txt'))) as f:
    lines = [line.rstrip() for line in f]
    for image in lines:
        buffer.append(os.path.split(image)[0])
        categories = list( dict.fromkeys(buffer) )

if intermediateResults==True:
    print(convLayers)

In [ ]:
# Save maximal value of the convolutional images for each convolution layer

dfFeatureMap = pd.DataFrame({"Layer":[], "filtre" : [], "image":[], "category":[], "maxima":[]}, dtype=int)      

# Create "n" submodels for "n" existing convolution layers
for layer in convLayers["convolutionIndex"]:
    print("Layer number ", layer)
    numberOfFilter = len(model.layers[layerNumber[layer-1]].get_weights()[1])
    modelLayer = Model(inputs=model.inputs, outputs=model.layers[layerNumber[layer-1]].output) # Create the submodel
    
    for category in tqdm(categories):  # For each folder, create a number for each category
        class_num = categories.index(category) 
        for line in lines:
            trueCategory = os.path.split(line)[0]
            if category == trueCategory:
                try:
                    imageName=os.path.split(line)[1]
                    img_array = cv2.imread(os.path.join(Path(dtd,category,os.path.split(line)[1])))  # open the image in grayscale
                    img_array= cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                    img_array = expand_dims(img_array, axis=0)                    
                    for filterNumber in range(numberOfFilter):
                        feature_maps = modelLayer.predict(img_array)
                        dfFeatureMap=dfFeatureMap.append({"Layer": layer,"filtre" : filterNumber, "image":imageName, "category":category, "maxima":np.max(feature_maps[0, :, :, filterNumber])}, ignore_index=True)
                except Exception as e:  # To keep the output clean
                    pass
    dfFeatureMap.to_csv(Path(os.path.join(outputPath,os.path.splitext(modelName)[0]+"_dfFeatureMap.csv")),index=True)
print(dfFeatureMap)
print("Done!")

In [ ]:
# Find the optimized value mutual information threshold for each filter

binsValue=30

dfFeatureMap = pd.read_csv(Path(os.path.join(storePath,os.path.splitext(modelName)[0]+"_dfFeatureMap.csv")))
iconfusionMatrix=pd.DataFrame({"Layer":[], "filter":[],"category":[], "iThreshold":[], "truePositive":[],"trueNegative":[], "falsePositive":[], "falsenegative":[], "mutualScore":[], "iInfoMutuelle":[]}, dtype=int)

for layer in convLayers["convolutionIndex"]:
    print("Layer number : ", layer)    
    numberOfFilter = len(model.layers[layerNumber[layer-1]].get_weights()[1])
    
    for filterNumber in range (numberOfFilter):
        print("filter number : ",filterNumber)
        for category in categories:
            print("category : ",category)
            #dfinodefect=dfFeatureMap[(dfFeatureMap['category']=="nodefect") & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber)]
            #dfidefect=dfFeatureMap[(dfFeatureMap['category']=="defect") & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber)]
            minND=np.amin(dfFeatureMap[(dfFeatureMap['category']==category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber)]["maxima"])
            maxND=np.amax(dfFeatureMap[(dfFeatureMap['category']==category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber)]["maxima"])
            thresholds=np.arange(minND, maxND, (maxND-minND)/20)
            for iThreshold in (thresholds):
                iTP=dfFeatureMap[(dfFeatureMap['category']==category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]>=iThreshold)].count()[4]
                iTN=dfFeatureMap[(dfFeatureMap['category']!=category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]<iThreshold)].count()[4]
                iFP=dfFeatureMap[(dfFeatureMap['category']!=category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]>=iThreshold)].count()[4]
                iFN=dfFeatureMap[(dfFeatureMap['category']==category) & (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]<iThreshold)].count()[4]
                iS=iTP+iTN+iFP+iFN
                label_true=[*np.ones(iTP),*np.zeros(iTN),*np.zeros(iFP),*np.ones(iFN)]
                label_pred=[*np.ones(iTP),*np.zeros(iTN),*np.ones(iFP),*np.zeros(iFN)]
                
                imutualScore=normalized_mutual_info_score(label_true, label_pred)
                iPY1 = (iTP+iFP)/iS
                iPY0 = (iFN+iTN)/iS
                iPC1 = (iTP+iFN)/iS
                iPC0 = (iFP+iTN)/iS
                iPY1C1, iPY0C1, iPY1C0, iPY0C0 = iTP/iS, iFN/iS, iFP/iS, iTN/iS
                
                if (iFP!=0 and iFN!=0 and iTP!=0 and iTN!=0):
                    iHY = - iPY1*math.log2(iPY1) - iPY0*math.log2(iPY0)
                    iHC = - iPC1*math.log2(iPC1) - iPC0*math.log2(iPC0)
                    iHYC = - iPY1C1*math.log2(iPY1C1) - iPY0C1*math.log2(iPY0C1) - iPY1C0*math.log2(iPY1C0) - iPY0C0*math.log2(iPY0C0)
                    iIYC = iHY + iHC - iHYC
                    iconfusionMatrix=iconfusionMatrix.append({"Layer":layer, "filter":filterNumber,"category":category, "iThreshold":iThreshold, "truePositive":iTP,"trueNegative":iTN, "falsePositive":iFP, "falsenegative":iFN, "mutualScore":imutualScore, "iInfoMutuelle":iIYC}, ignore_index=True)        
                else :
                    iconfusionMatrix=iconfusionMatrix.append({"Layer":layer, "filter":filterNumber,"category":category, "iThreshold":iThreshold, "truePositive":iTP,"trueNegative":iTN, "falsePositive":iFP, "falsenegative":iFN, "mutualScore":np.NaN, "iInfoMutuelle":np.NaN}, ignore_index=True)
    iconfusionMatrix.to_csv(Path(os.path.join(outputPath,os.path.splitext(modelName)[0]+"_iconfusionMatrix.csv")),index=True)
print("Done!")

In [ ]:
# Built the confusion matrix with optimized value of mutual information for each filter
iconfusionMatrix = pd.read_csv(Path(os.path.join(storePath,os.path.splitext(modelName)[0]+"_iconfusionMatrix.csv")))
dfFeatureMap = pd.read_csv(Path(os.path.join(storePath,os.path.splitext(modelName)[0]+"_dfFeatureMap.csv")))
confusionMatrix=pd.DataFrame({"Layer":[], "filter":[], "category":[], "threshold":[], "truePositive":[],"trueNegative":[], "falsePositive":[], "falsenegative":[], "mutualScore":[], "mutualInfo":[]}, dtype=int)
layerNumber=[0,4,8]
for layer in convLayers["convolutionIndex"]:
    numberOfFilter = len(model.layers[layerNumber[layer-1]].get_weights()[1])
    print(layer)
    for filterNumber in range (numberOfFilter):
        print(filterNumber)
        for category in categories:
            threshold=iconfusionMatrix.loc[(iconfusionMatrix["category"]==category)&(iconfusionMatrix["Layer"]==layer)&(iconfusionMatrix["filter"]==filterNumber)].sort_values(by="iInfoMutuelle", ascending=False).iloc[0]["iThreshold"]
            
            TP=dfFeatureMap[(dfFeatureMap['category']==category) &  (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]>=threshold)].count()[4]
            TN=dfFeatureMap[(dfFeatureMap['category']!=category) &  (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]<threshold)].count()[4]
            FP=dfFeatureMap[(dfFeatureMap['category']!=category) &  (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]>=threshold)].count()[4]
            FN=dfFeatureMap[(dfFeatureMap['category']==category) &  (dfFeatureMap['Layer']==layer) & (dfFeatureMap['filtre']==filterNumber) & (dfFeatureMap["maxima"]<threshold)].count()[4]
            S=TP+TN+FP+FN
            
            # Calcul of the mutual information
            PY1 = (TP+FP)/S
            PY0 = (FN+TN)/S
            PC1 = (TP+FN)/S
            PC0 = (FP+TN)/S 
            PY1C1, PY0C1, PY1C0, PY0C0 = TP/S, FN/S, FP/S, TN/S
            
            HY = - PY1*math.log2(PY1) - PY0*math.log2(PY0)
            HC = - PC1*math.log2(PC1) - PC0*math.log2(PC0)
            HYC = - PY1C1*math.log2(PY1C1) - PY0C1*math.log2(PY0C1) - PY1C0*math.log2(PY1C0) - PY0C0*math.log2(PY0C0)     
            IYC = HY + HC - HYC
        
            label_true=[*np.ones(TP),*np.zeros(TN),*np.zeros(FP),*np.ones(FN)]
            label_pred=[*np.ones(TP),*np.zeros(TN),*np.ones(FP),*np.zeros(FN)]
            mutualScore=normalized_mutual_info_score(label_true, label_pred)
    
            confusionMatrix=confusionMatrix.append({"Layer":layer, "filter":filterNumber,"category":category, "threshold":threshold, "truePositive":TP,"trueNegative":TN, "falsePositive":FP, "falsenegative":FN, "mutualScore":mutualScore, "mutualInfo":IYC}, ignore_index=True)

confusionMatrix.to_csv(Path(os.path.join(storePath,os.path.splitext(modelName)[0]+"_confusionMatrix.csv")),index=True)


In [ ]:
# Plot mutual information (last decile) through the Newkork (sorted) - Pandas version - DRAFT

#confusionMatrix = pd.read_csv("/home/quentin/Documents/NIPS/2_pipeline/store/20200405_162613_exp2_BS64_confusionMatrix.csv")
layerToSort=0
viridis = cm.get_cmap('viridis', 12)
fig, ax = plt.subplots(1,1,figsize=(pageWidth,7))
i=0.1
layers = [1,2,3,4,5]
tab=pd.DataFrame({"layer":[],"category":[], "decile":[], "category":[]})
SMALL_SIZE=9
plt.rc('xtick', labelsize=SMALL_SIZE)
for layer in layers:
    for category in categories:    
        tab=tab.append({"layer":layer, "category":category, "decile":np.percentile(confusionMatrix[(confusionMatrix["Layer"]==layer)&(confusionMatrix["category"]==category)]["mutualInfo"],np.arange(0,100,10))[9], "category":category},ignore_index=True)

x=[]
y=[]
for layer in (layers):
    xDecile=np.array(tab[tab["layer"]==layer]["decile"])
    yCategory=np.array(tab[tab["layer"]==layer]["category"])
    x=np.append(x,xDecile,axis=0)
    y=np.append(y,yCategory,axis=0)
x=x.reshape(5,47)
y=y.reshape(5,47) #test
g=x[:,np.argsort(x[0])]
sortedCategory=y[0,np.argsort(x[layerToSort])] #test
im = ax.imshow(g)
ax.set_yticks(np.arange(0,len(layers)))
ax.set_yticklabels(layers)
ax.set_xticks(np.arange(len(categories)))
ax.set_xticklabels(sortedCategory)
ax.set_ylabel('Layers')
ax.set_title("Mutual information score through the network (last decile)")
ax.vlines(np.arange(-0.5,len(categories)-1), -.5, len(layers)-.5, 'w')
plt.setp(ax.get_xticklabels(), rotation=90, va="center", ha="right",
         rotation_mode="anchor")
cbar=fig.colorbar(im,ax=ax, pad=0.01, aspect=5, ticks=[0,0.01,0.02], shrink=0.22)
cbar.set_ticks([0,0.01,0.02])
cbar.set_ticklabels([0,0.01,0.02])
plt.tight_layout()
#plt.savefig("/home/quentin/Documents/NIPS/2_pipeline/store/20200405_162613_exp2_BS64_MILastDecile.pdf")

plt.show()

# clustering

In [ ]:
dataDirectory="C:/Users/AQ62270/201910_ccn/NIPS/0_data/dtd/images/spiralled/"
confusionMatrix = pd.read_csv(Path(os.path.join("C:/Users/AQ62270/201910_ccn/NIPS/2_pipeline/store/20200405_162613_exp2_BS64_confusionMatrix.csv")))


In [ ]:

#modelName="20200405_162613_exp2_BS64.model"
inputDirectory=Path("2_pipeline/store/") # Location of the model
inputPath=Path(projectDirectory/inputDirectory)
modelPath=os.path.join(inputPath,modelName)
model_5layers = keras.models.load_model(os.path.join(Path(projectDirectory/inputDirectory),"20200405_162613_exp2_BS64.model"))
model_3layers = keras.models.load_model(os.path.join(Path(projectDirectory/inputDirectory),"20200507_004956_exp2_BS64_3layers.model"))

model_5layers_1stConv=Model(inputs=model_5layers.inputs, outputs=model_5layers.layers[layerNumber[layer-1]].output)
model_3layers_1stConv=Model(inputs=model_3layers.inputs, outputs=model_3layers.layers[layerNumber[layer-1]].output)

model_5layers_1stConv.summary()
model_3layers_1stConv.summary()

In [ ]:
img_array=cv2.imread(os.path.join(Path("C:/Users/AQ62270/201910_ccn/NIPS/0_data/dtd/images/spiralled/spiralled_0004.jpg")))
img_array = cv2.resize(img_array, (227, 227))  # resize the image
plt.imshow(new_array[:,:,0])
img_array.shape

In [ ]:
blur11 = cv2.GaussianBlur(feature_maps[0, :, :,filterNumber],(11,11),0)
blur7 = cv2.GaussianBlur(feature_maps[0, :, :,filterNumber],(7,7),0)
DoGimg = blur10 - blur7
plt.imshow(blur11)
plt.show()
plt.imshow(blur7)
plt.show()
plt.imshow(DoGimg)
plt.show()

In [ ]:
# Plot filter or associated feature maps / model CNN2
layer=1

img_array=cv2.imread(os.path.join(Path("C:/Users/AQ62270/201910_ccn/NIPS/0_data/dtd/images/spiralled/spiralled_0004.jpg")))
img_array = cv2.resize(img_array, (227, 227))  # resize the image
img_array = expand_dims(img_array, axis=0)
plt.imshow(new_array)
plt.show()

cols=8
rows=12
fig,ax = plt.subplots(nrows=rows, ncols=cols, figsize=(15,20))
filterNumber=0
for i in np.arange(0,cols):
    for j in np.arange(0,rows):
        # plot feature maps or...
        #feature_maps = model_5layers_1stConv.predict(img_array)
        #ax[j,i].imshow(feature_maps[0, :, :,filterNumber], cmap='gray')
        
        # ...plot filters (comment last paragraph)
        ax[j,i].imshow(model_5layers_1stConv.layers[layerNumber[layer-1]+1].get_weights()[0][:,:,0, filterNumber], cmap='gray')
        
        filterNumber+=1
plt.show()

In [ ]:
# Plot filter or associated feature maps / model CNN3

cols=8
rows=12
fig,ax = plt.subplots(nrows=rows, ncols=cols, figsize=(15,20))
filterNumber=0
for i in np.arange(0,cols):
    for j in np.arange(0,rows):
        # plot feature maps or...
        #feature_maps = model_3layers_1stConv.predict(img_array)
        #ax[j,i].imshow(feature_maps[0, :, :,filterNumber], cmap='gray')
        
        # ...plot filters (comment last paragraph)
        ax[j,i].imshow(model_3layers_1stConv.layers[layerNumber[layer-1]+1].get_weights()[0][:,:,0, filterNumber], cmap='gray')
        
        filterNumber+=1
plt.show()

In [ ]:
feature_maps.shape
print(feature_maps[0,0,1,:])

for i in  (np.arange(0,55))[:2]:
    for j in  (np.arange(0,55))[:2]:
        for filterNumber in np.arange(0,96):
            #print(feature_maps[0,i,j,:])
            blur11 = cv2.GaussianBlur(feature_maps[0, j, i,filterNumber],(11,11),0)
            blur7 = cv2.GaussianBlur(feature_maps[0, j, i,filterNumber],(7,7),0)
            DoGimg = blur10 - blur7
            ax[j,i].imshow(DoGimg)
plt.show()
#        feature_maps = model_3layers_1stConv.predict(img_array)
#        plt.imshow(feature_maps[0, :, :,filterNumber], cmap='gray')
#        filterNumber+=1
#    plt.show()

In [ ]:
feature_maps[0, :, :,filterNumber]

In [ ]:


blur11 = cv2.GaussianBlur(feature_maps[0, :, :,filterNumber],(11,11),0)
blur7 = cv2.GaussianBlur(feature_maps[0, :, :,filterNumber],(7,7),0)
DoGimg = blur10 - blur7
plt.imshow(blur11)
plt.show()
plt.imshow(blur7)
plt.show()
plt.imshow(DoGimg)
plt.show()

In [ ]:
confusionMatrix[(confusionMatrix["Layer"]==1)].sort_values(by="mutualInfo", ascending=False)

In [ ]:
img_array=cv2.imread(os.path.join(Path("C:/Users/AQ62270/201910_ccn/NIPS/0_data/dtd/images/spiralled/spiralled_0004.jpg")))
new_array = cv2.resize(img_array, (227, 227))  # resize the image
plt.imshow(new_array)
plt.show()

In [ ]:
filteredImage = cv2.GaussianBlur(,(5,5),0)